In [67]:
import numpy as np
from matplotlib import pyplot as pl

import scipy.io.wavfile as wav
from scikits.talkbox.features import mfcc

%matplotlib inline

import os
BASE_DIR = 'vowels/'
files = os.listdir(BASE_DIR)

# 7.1

In [68]:
nafFiles = []
namFiles = []

# Get all natural female and male wav files by itering all the files' names.
# The first value of each array's elements is the simple rate, while the second one is the values.
for f in files:
    if f.startswith("naf"):
        nafFiles.append(wav.read(BASE_DIR + f))
    if f.startswith("nam"):
        namFiles.append(wav.read(BASE_DIR + f))

In [71]:
nafCeps = []
for f in nafFiles:
    nafCeps.append(mfcc(f[1], fs=f[0])[0])

namCeps = []
for f in namFiles:
    namCeps.append(mfcc(f[1], fs=f[0])[0])

In [74]:
nafMeanCeps
for c in nafCeps:
    nafMeanCeps = np.mean(c, axis=0)
    print nafMeanCeps

[ 15.59676448   2.80003934  -0.39652586  -0.76364244  -1.25470778
   0.79151527  -0.43268339   0.10226868   0.04713486   0.31112434
   0.18058686  -0.06080008   0.17647303]
[ 15.25868063   1.55432464   0.47707232   1.58101891  -0.60629484
  -0.41603403  -0.57384826   0.28345123  -0.35312312  -0.0234101
   0.12874776   0.34592014  -0.29469729]
[ 13.90867852   0.68129169   2.05023316   1.69824959  -1.11848276
   1.1193401    0.36488438  -0.02601204  -0.36810946  -0.303512
  -0.27483869  -0.06704484   0.02052806]
[ 15.80234939   2.04174719  -0.3284331    0.05370534   0.28048581
   0.02193069  -0.23488165  -0.54157618  -0.11982253   0.42491178
  -0.17147618  -0.58432596   0.56458697]
[ 14.61531825   0.62442115   0.14397496   2.29468211   0.99964086
  -0.06981434  -0.28476776   0.04074258  -0.85627847  -0.42439235
  -0.22276625  -0.51432127   0.42661653]
[  1.56220512e+01   2.16217379e+00  -6.31645247e-01  -5.52234332e-01
  -7.37977930e-01   8.63676695e-01  -1.96047121e-01   3.14027768e-03


In [39]:
namMeanCeps = np.mean(namCeps[0], axis=0)
print namMeanCeps

[ 13.67969351   1.50603074   0.78396184   0.4113546    0.0454019
   1.10128161   0.96236364  -0.77496485   0.16466424  -0.43918445
   0.8500918   -0.84605079   0.06960457]


# 7.2

In [40]:
nkFiles = []

for f in files:
    if f.startswith("nk"):
        nkFiles.append(wav.read(BASE_DIR + f))

In [41]:
nkCeps = []
for f in nkFiles:
    nkCeps.append(mfcc(f[1], fs=f[0])[0])

In [42]:
print namMeanCeps

[ 14.80595194   0.92440865   2.66664348   0.78685828  -1.12664115
   0.1401904   -0.47762766  -0.81662582   0.30184514  -0.14245167
   0.05061334   0.09102493   0.3083708 ]


In [43]:
nkMeanCeps = np.mean(nkCeps[0], axis=0)
print nkMeanMeanCeps

[ 15.25868063   1.55432464   0.47707232   1.58101891  -0.60629484
  -0.41603403  -0.57384826   0.28345123  -0.35312312  -0.0234101
   0.12874776   0.34592014  -0.29469729]


# 7.3

In [61]:
print nafMeanCeps

[ 15.59676448   2.80003934  -0.39652586  -0.76364244  -1.25470778
   0.79151527  -0.43268339   0.10226868   0.04713486   0.31112434
   0.18058686  -0.06080008   0.17647303]


In [62]:
print namMeanCeps

[ 13.67969351   1.50603074   0.78396184   0.4113546    0.0454019
   1.10128161   0.96236364  -0.77496485   0.16466424  -0.43918445
   0.8500918   -0.84605079   0.06960457]


# 7.4

In [64]:
print namMeanCeps

[ 15.59676448   2.80003934  -0.39652586  -0.76364244  -1.25470778
   0.79151527  -0.43268339   0.10226868   0.04713486   0.31112434
   0.18058686  -0.06080008   0.17647303]


In [65]:
print nafMeanCeps

[ 15.59676448   2.80003934  -0.39652586  -0.76364244  -1.25470778
   0.79151527  -0.43268339   0.10226868   0.04713486   0.31112434
   0.18058686  -0.06080008   0.17647303]


In [66]:
print nkMeanCeps

NameError: name 'nkMeanCeps' is not defined

# 7.5

In [49]:
naturalFiles = []
synthesizedFiles = []

for f in files:
    if "n" in f:
        naturalFiles.append(wav.read(BASE_DIR + f))
    if "s" in f:
        synthesizedFiles.append(wav.read(BASE_DIR + f))

In [52]:
naturalCeps = []
for f in naturalFiles:
    naturalCeps.append(mfcc(f[1], fs=f[0])[0])

synthesizedCeps = []
for f in synthesizedFiles:
    synthesizedCeps.append(mfcc(f[1], fs=f[0])[0])

In [53]:
naturalMeanCeps = np.mean(naturalCeps[0], axis=0)
print naturalMeanCeps

[ 15.59676448   2.80003934  -0.39652586  -0.76364244  -1.25470778
   0.79151527  -0.43268339   0.10226868   0.04713486   0.31112434
   0.18058686  -0.06080008   0.17647303]


In [54]:
synthesizedMeanCeps = np.mean(synthesizedCeps[0], axis=0)
print synthesizedMeanCeps

[ 13.70235356   3.14177536  -1.31348435   2.01924182   0.64619545
  -0.31214544  -0.15833591   0.0560468    0.03908075  -0.39810772
  -0.28138121  -0.1422775   -0.01718017]


# 7.6 7yo vs 3yo

In [56]:
_3yoFiles = []
_7yoFiles = []

for f in files:
    if f.startswith("nk3"):
        _3yoFiles.append(wav.read(BASE_DIR + f))
    if f.startswith("nk7"):
        _7yoFiles.append(wav.read(BASE_DIR + f))

In [57]:
_3yoCeps = []
for f in _3yoFiles:
    _3yoCeps.append(mfcc(f[1], fs=f[0])[0])

_7yoCeps = []
for f in _7yoFiles:
    _7yoCeps.append(mfcc(f[1], fs=f[0])[0])

In [59]:
_3yoMeanCeps = np.mean(_3yoCeps[0], axis=0)
print _3yoMeanCeps

[ 16.79714949   1.54090856   0.11806946   0.75369388  -1.12860447
  -0.07059938   0.15398047   0.6023547   -0.39026959   0.66859702
   0.11442308  -0.02079662   0.25363452]


In [60]:
_7yoMeanCeps = np.mean(_7yoCeps[0], axis=0)
print _7yoMeanCeps

[  1.56494220e+01   1.81918608e+00  -6.28691171e-01  -6.50586184e-01
  -5.23806822e-01   7.55097353e-01   1.78484449e-01   3.07568984e-01
   9.78368565e-02  -2.11253999e-02   8.12625526e-02  -1.06741066e-02
   1.63699384e-01]
